<a href="https://colab.research.google.com/github/Tuevu110405/AIO_Module_7/blob/feature%2Ftraining/detection2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class FirstFeature(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FirstFeature, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, 1,0, bias = False),
            nn.LeakyReLu()
        )

    def forward(self, x):
        return self.conv(x)

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1,1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLu(inplace = True),
            nn.Conv2d(out_channels, out_channels, 3, 1,1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLu(inplace = True)
        )

    def forward(self, x):
        return self.conv(x)

class Encoder(nn.Module):
    def __init__(self, in_channels, out_channels) -> None:
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.MaxPool2d(2),
            ConvBlock(in_channels, out_channels)
        )

    def forward(self, x):
        return self.encoder(x)

class Decoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Decoder, self).__init__()
        self.conv = nn.Sequential(
            nn.UpsamplingBilinear2d(scale_factor = 2),
            nn.Conv2d(in_channels, out_channels, 1, 1, 0, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLu(inplace = True)
        )
        self.conv_block = ConvBlock(in_channels, out_channels)

    def forward(self, x, skip):
        x = self.conv(x)
        x = torch.concat([x, skip], dim = 1)
        x = self.conv_block(x)
        return x

class FinalOutput(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FinalOutput, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, 1, 0, bias = False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.conv(x)

class Unet(nn.Module):
    def __init__(
            self, n_channels = 3, n_classes = 3, features = [64, 128, 256, 512]
    ):
        super(Unet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.in_conv1 = FirstFeature(n_channels, features[0])
        self.in_conv2 = ConvBlock(features[0], features[0])

        self.enc_1 = Encoder(features[0], features[1])
        self.enc_2 = Encoder(features[1], features[2])
        self.enc_3 = Encoder(features[2], features[3])
        self.enc_4 = Encoder(features[3], 1024)

        self.dec_1 = Decoder(1024, features[3])
        self.dec_2 = Decoder(features[3], features[2])
        self.dec_3 = Decoder(features[2], features[1])
        self.dec_4 = Decoder(features[1], features[0])

        self.out_conv = FinalOutput(features[0], n_classes)

    def forward(self, x):
        x = self.in_conv1(x)
        x1 = self.in_conv2(x)

        x2 = self.enc_1(x1)
        x3 = self.enc_2(x2)
        x4 = self.enc_3(x3)
        x5 = self.enc_4(x4)

        x = self.dec_1(x5, x4)
        x = self.dec_2(x, x3)
        x = self.dec_3(x, x2)
        x = self.dec_4(x, x1)

        x = self.out_conv(x)
        return x

Super Resolution task

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, img_dir, is_train = True):
        self.resize = transforms.Resize((LOW_IMG_WIDTH, LOW_IMG_HEIGHT),
                                        antialias = True)
        self.is_train = is_train
        self.img_dir = img_dir
        self.images = os.listdir(img_dir)

    def __len__(self):
        return len(self.images)

    def normalize(self, input_image, target_image):
        input_image = input_image * 2 -1
        target_image = target_image * 2 -1
        return input_image, target_image

    def random_jitter(self, input_image, target_image):
        if torch.rand([]) < 0.5:
            input_image = transforms.functional.hflip(input_image)
            target_image = transforms.functional.hflip(target_image)

        return input_image, target_image

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        image = np.array(Image.open(img_path)).convert("RGB")
        image = transforms.functional.to_tensor(image)
        input_image = self.resize(image).type(torch.float32)
        target_image = image.type(torch.float32)

        if self.is_train:
            input_image, target_image = self.random_jitter(input_image, target_image)

        return input_image, target_image



In [ ]:
class FirstFeature(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FirstFeature, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, 1,0, bias = False),
            nn.LeakyReLu()
        )

    def forward(self, x):
        return self.conv(x)

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1,1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLu(inplace = True),
            nn.Conv2d(out_channels, out_channels, 3, 1,1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLu(inplace = True)
        )

    def forward(self, x):
        return self.conv(x)

class Encoder(nn.Module):
    def __init__(self, in_channels, out_channels) -> None:
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.MaxPool2d(2),
            ConvBlock(in_channels, out_channels)
        )

    def forward(self, x):
        return self.encoder(x)

class Decoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Decoder, self).__init__()
        self.conv = nn.Sequential(
            nn.UpsamplingBilinear2d(scale_factor = 2),
            nn.Conv2d(in_channels, out_channels, 1, 1, 0, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLu(inplace = True)
        )
        self.conv_block = ConvBlock(in_channels, out_channels)

    def forward(self, x, skip):
        x = self.conv(x)
        x = torch.concat([x, skip], dim = 1)
        x = self.conv_block(x)
        return x

class FinalOutput(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FinalOutput, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, 1, 0, bias = False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.conv(x)

class Unet(nn.Module):
    def __init__(
            self, n_channels = 3, n_classes = 3, features = [64, 128, 256, 512]
    ):
        super(Unet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.resize = transforms.Resize((LOW_IMG_WIDTH * 4, LOW_IMG_HEIGHT *4),
                                        antialias = True)

        self.in_conv1 = FirstFeature(n_channels, features[0])
        self.in_conv2 = ConvBlock(features[0], features[0])

        self.enc_1 = Encoder(features[0], features[1])
        self.enc_2 = Encoder(features[1], features[2])
        self.enc_3 = Encoder(features[2], features[3])
        self.enc_4 = Encoder(features[3], 1024)

        self.dec_1 = Decoder(1024, features[3])
        self.dec_2 = Decoder(features[3], features[2])
        self.dec_3 = Decoder(features[2], features[1])
        self.dec_4 = Decoder(features[1], features[0])

        self.out_conv = FinalOutput(features[0], n_classes)

    def forward(self, x):
        x = self.in_conv1(x)
        x1 = self.in_conv2(x)

        x2 = self.enc_1(x1)
        x3 = self.enc_2(x2)
        x4 = self.enc_3(x3)
        x5 = self.enc_4(x4)

        x = self.dec_1(x5, x4)
        x = self.dec_2(x, x3)
        x = self.dec_3(x, x2)
        x = self.dec_4(x, x1)

        x = self.out_conv(x)
        return x

Inpaint

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, img_dir, is_train = True):
        self.is_train = is_train
        self.img_dir = img_dir
        self.images = os.listdir(img_dir)

    def __len__(self):
        return len(self.images)

    def normalize(self, input_image, target_image):
        input_image = input_image * 2 -1
        target_image = target_image * 2 -1
        return input_image, target_image

    def random_jitter(self, input_image, target_image):
        if torch.rand([]) < 0.5:
            input_image = transforms.functional.hflip(input_image)
            target_image = transforms.functional.hflip(target_image)

        return input_image, target_image

    def create_mask(self, image):
        masked_image = image.copy()
        #Prepare masking matrix
        mask = np.full((IMG_WIDTH, IMG_HEIGHT,3), 0, np.uint8)
        for _ in range(np.random.randint(1,5)):
            x1, x2 = np.random.randint(1, IMG_WIDTH), np.random.randint(1, IMG_WIDTH)
            y1, y2 = np.random.randint(1, IMG_HEIGHT), np.random.randint(1, IMG_HEIGHT)

            thickess = np.random.randint(1,15)
            cv2.line(mask, (x1,y1), (x2,y2), (1, 1, 1), thickess)

        masked_image = np.where(mask, 255*np.ones_like(mask), masked_image)
        return masked_image

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        image = np.array(Image.open(img_path).convert("RGB"))

        input_image = self.create_mask(image)
        input_image = transforms.functional.to_tensor(input_image)
        target_image = transforms.functional.to_tensor(image)

        input_image = input_image.type(torch.float32)
        target_image = target_image.type(torch.float32)

        input_image, target_image = self.normalize(input_image, target_image)

        if self.is_train:
            input_image, target_image = self.random_jitter(input_image, target_image)

        return input_image, target_image


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
